# Data 5000:  Testing and handling data

Prediction public transport delays in the TTC Toronto's public transport.

Author: Marc Vicuna

Last update: March 2nd 2022.

### Modules

In [ ]:
# Python utility
import os
from datetime import datetime
import re

# Data processing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

# Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from keras.models import load_model

#Testing
from sklearn.metrics import confusion_matrix, f1_score, mean_squared_error

## Data Import
Downloads from https://open.toronto.ca/dataset/ttc-subway-delay-data/ and https://open.toronto.ca/dataset/ttc-bus-delay-data/

In [1]:
buslist = []
subwaylist = []
eastlist = []
centerlist = []
dfbus= pd.DataFrame()
dfsubway= pd.DataFrame()
dfeast= pd.DataFrame()
dfcenter= pd.DataFrame()
for root, dirs, files in os.walk("."):
    for file in files: #matching folder
        st = os.path.join(root,file)
        if st[-5:]==".xlsx" and "\\bus_data\\" in st:
            buslist.append(st)
        elif st[-5:]==".xlsx" and "\\subway_data\\" in st:
            subwaylist.append(st)
        elif st[-4:]==".csv" and "\\East (Airport)\\" in st:
            eastlist.append(st)
        elif st[-4:]==".csv" and "\\City Centre\\" in st:
            centerlist.append(st)
for f in buslist:
    dfbus=pd.concat([dfbus,pd.concat(pd.read_excel(f, engine='openpyxl', index_col=None, sheet_name=None),sort=False)],sort=False)
for f in subwaylist:
    dfsubway=pd.concat([dfsubway,pd.concat(pd.read_excel(f, engine='openpyxl', index_col=None, sheet_name=None),sort=False)],sort=False)
for f in eastlist:
    dfeast=pd.concat([dfeast,pd.read_csv(f, index_col=None)],sort=False)
for f in centerlist:
    dfcenter=pd.concat([dfcenter,pd.read_csv(f, index_col=None)],sort=False)

## Data cleaning
Both East and Center weather for Toronto are considered, in favour of the East weather as the lower concentration of human activity makes in more sensible to 

In [2]:
# Cleaning dfbus
dfbus.loc[dfbus["Date"].isna(),"Date"]=dfbus[dfbus["Date"].isna()]["Report Date"]
dfbus.loc[dfbus["Delay"].isna(),"Delay"]=dfbus[dfbus["Delay"].isna()]["Min Delay"]
dfbus.loc[dfbus["Gap"].isna(),"Gap"]=dfbus[dfbus["Gap"].isna()]["Min Gap"]
dfbus.loc[dfbus["Route"].isna(),"Route"]=dfbus[dfbus["Route"].isna()]["Line"]
dfbus.loc[dfbus["Direction"].isna(),"Direction"]=dfbus[dfbus["Direction"].isna()]["Bound"]
dfbus.drop("Report Date", axis=1, inplace=True)
dfbus.drop("Min Gap", axis=1, inplace=True)
dfbus.drop("Line", axis=1, inplace=True)
dfbus.drop("Bound", axis=1, inplace=True)
dfbus = dfbus.drop(dfbus[dfbus["Date"].isna()].index)
dfbus.reset_index(inplace=True)
dfbus.drop("Incident ID", axis=1, inplace=True)
dfbus.drop("level_1", axis=1, inplace=True)
dfbus.drop("level_0", axis=1, inplace=True)
dfbus.drop("Min Delay", axis=1, inplace=True)
dfbus["Day"]=dfbus["Date"].apply(lambda x: int(x.strftime("%d")))
dfbus["Month"]=dfbus["Date"].apply(lambda x: int(x.strftime("%m")))
dfbus["Year"]=dfbus["Date"].apply(lambda x: int(x.strftime("%Y")))
dfbus.drop("Date", axis=1, inplace=True)
dfbus["Hour"]=dfbus["Time"].apply(lambda x: int(x[0:2]) if isinstance(x, str) else x.hour)
dfbus["Minute"]=dfbus["Time"].apply(lambda x: int(x[3:5]) if isinstance(x, str) else x.minute)
dfbus.drop("Time", axis=1, inplace=True)
dfbus.drop("Location", axis=1, inplace=True)
dfbus.drop(" Min Delay", axis=1, inplace=True)

# Cleaning dfsubway
dfsubway.reset_index(inplace=True)
dfsubway.drop("level_0", axis=1, inplace=True)
dfsubway.drop("level_1", axis=1, inplace=True)
dfsubway.drop("Unnamed: 10", axis=1, inplace=True)
dfsubway.drop("Unnamed: 11", axis=1, inplace=True)
dfsubway.drop("Unnamed: 12", axis=1, inplace=True)
dfsubway["Day"]=dfsubway["Date"].apply(lambda x: int(x.strftime("%d")))
dfsubway["Month"]=dfsubway["Date"].apply(lambda x: int(x.strftime("%m")))
dfsubway["Year"]=dfsubway["Date"].apply(lambda x: int(x.strftime("%Y")))
dfsubway.drop("Date", axis=1, inplace=True)
dfsubway["Hour"]=dfsubway["Time"].apply(lambda x: int(x[0:2]))
dfsubway["Minute"]=dfsubway["Time"].apply(lambda x: int(x[3:5]))
dfsubway.drop("Time", axis=1, inplace=True)

# Clearing dfeast
dfeast.drop("Visibility Flag", axis=1, inplace=True)
dfeast.drop("Wind Spd Flag", axis=1, inplace=True)
dfeast.drop("Stn Press (kPa)", axis=1, inplace=True)
dfeast.drop("Stn Press Flag", axis=1, inplace=True)
dfeast.drop("Hmdx", axis=1, inplace=True)
dfeast.drop("Wind Chill Flag", axis=1, inplace=True)
dfeast.drop("Wind Chill", axis=1, inplace=True)
dfeast.drop("Date/Time (LST)", axis=1, inplace=True)
dfeast.drop("Temp Flag", axis=1, inplace=True)
dfeast.drop("Dew Point Temp Flag", axis=1, inplace=True)
dfeast.drop("Rel Hum Flag", axis=1, inplace=True)
dfeast.drop("Precip. Amount (mm)", axis=1, inplace=True)
dfeast.drop("Precip. Amount Flag", axis=1, inplace=True)
dfeast.drop("Wind Dir Flag", axis=1, inplace=True)
dfeast.drop("Hmdx Flag", axis=1, inplace=True)
dfeast.drop("Station Name", axis=1, inplace=True) #Toronto Intl A
dfeast.drop("Climate ID", axis=1, inplace=True)
dfeast.drop("Longitude (x)", axis=1, inplace=True) #-79.4,43.68
dfeast.drop("Latitude (y)", axis=1, inplace=True)
dfeast["Hour"]=dfeast["Time (LST)"].apply(lambda x: int(x[0:2]))
dfeast.drop("Time (LST)", axis=1, inplace=True)

# Clearing dfcenter
dfcenter.drop("Visibility Flag", axis=1, inplace=True)
dfcenter.drop("Wind Spd Flag", axis=1, inplace=True)
dfcenter.drop("Stn Press (kPa)", axis=1, inplace=True)
dfcenter.drop("Stn Press Flag", axis=1, inplace=True)
dfcenter.drop("Hmdx", axis=1, inplace=True)
dfcenter.drop("Wind Chill Flag", axis=1, inplace=True)
dfcenter.drop("Wind Chill", axis=1, inplace=True)
dfcenter.drop("Date/Time (LST)", axis=1, inplace=True)
dfcenter.drop("Temp Flag", axis=1, inplace=True)
dfcenter.drop("Dew Point Temp Flag", axis=1, inplace=True)
dfcenter.drop("Rel Hum Flag", axis=1, inplace=True)
dfcenter.drop("Precip. Amount (mm)", axis=1, inplace=True)
dfcenter.drop("Precip. Amount Flag", axis=1, inplace=True)
dfcenter.drop("Wind Dir Flag", axis=1, inplace=True)
dfcenter.drop("Hmdx Flag", axis=1, inplace=True)
dfcenter.drop("Station Name", axis=1, inplace=True) #Toronto Intl A
dfcenter.drop("Climate ID", axis=1, inplace=True)
dfcenter.drop("Longitude (x)", axis=1, inplace=True) #-79.63,43.63
dfcenter.drop("Latitude (y)", axis=1, inplace=True)
dfcenter["Hour"]=dfcenter["Time (LST)"].apply(lambda x: int(x[0:2]))
dfcenter.drop("Time (LST)", axis=1, inplace=True)

# Combining dataframes
weathersubway = pd.merge(dfsubway, dfeast, on=['Year','Month','Day','Hour'], how='left')
weathersubway=weathersubway.drop(weathersubway[weathersubway["Temp (°C)"].isna()].index) # removes 146 rows
weathersubway=weathersubway.drop(weathersubway[weathersubway["Dew Point Temp (°C)"].isna()].index)
weathersubway=weathersubway.drop(weathersubway[weathersubway["Visibility (km)"].isna()].index)
weathersubway=weathersubway.drop(weathersubway[weathersubway["Wind Spd (km/h)"].isna()].index)
weathersubway=weathersubway.drop(weathersubway[weathersubway["Rel Hum (%)"].isna()].index)
weathersubway=weathersubway.drop(weathersubway[weathersubway["Wind Dir (10s deg)"].isna()].index)

## Subway Data Cleaning

Initialisation of dictionnaries before the next step.

In [4]:
prediction_var = 'Code'

weathersubway['Weekday'] = weathersubway.apply(lambda row: datetime(row.Year, row.Month, row.Day).weekday(), axis=1)
weathersubway['Latitude'] = weathersubway.apply(lambda row: coords[row.Station][0], axis=1)
weathersubway['Longitude'] = weathersubway.apply(lambda row: coords[row.Station][1], axis=1)
weathersubway=weathersubway.drop(weathersubway[weathersubway["Longitude"].isna()].index) #500
weathersubway=weathersubway.drop(weathersubway[weathersubway["Line"].isna()].index) #500
weathersubway=weathersubway.drop(weathersubway[weathersubway["Code"].isna()].index) #1
weathersubway.drop("Station", axis=1, inplace=True)
#weathersubway.drop("Line", axis=1, inplace=True)
weathersubway.drop("Vehicle", axis=1, inplace=True)
weathersubway = weathersubway.fillna('na')

exit_code = {}
for a, i in enumerate(sorted(set(weathersubway[prediction_var]))):
    exit_code[i]=a
weathersubway[prediction_var]=weathersubway[prediction_var].map(exit_code)

weathersubway["Rain"]=weathersubway["Weather"].apply(lambda row: 1 if 'Rain' in row else 0)
weathersubway["Cloudy"]=weathersubway["Weather"].apply(lambda row: 1 if 'Cloudy' in row else 0)
weathersubway["Snow"]=weathersubway["Weather"].apply(lambda row: 1 if 'Snow' in row else 0)
weathersubway["Fog"]=weathersubway["Weather"].apply(lambda row: 1 if 'Fog' in row else 0)
weathersubway["Freezing"]=weathersubway["Weather"].apply(lambda row: 1 if 'Freezing' in row else 0)
weathersubway["Drizzle"]=weathersubway["Weather"].apply(lambda row: 1 if 'Drizzle' in row else 0)
weathersubway["Thunderstorms"]=weathersubway["Weather"].apply(lambda row: 1 if 'Thunderstorms' in row else 0)
weathersubway.drop("Weather", axis=1, inplace=True)

features_to_encode = weathersubway.columns[weathersubway.dtypes==object].tolist()
col_trans = make_column_transformer(
                        (OneHotEncoder(),features_to_encode),
                        remainder = "passthrough"
                        )

## Bus Data Cleaning

In [3]:
weatherbus = pd.merge(dfbus, dfeast, on=['Year','Month','Day','Hour'], how='left')
weatherbus=weatherbus.drop(weatherbus[weatherbus["Temp (°C)"].isna()].index)
weatherbus=weatherbus.drop(weatherbus[weatherbus["Dew Point Temp (°C)"].isna()].index)
weatherbus=weatherbus.drop(weatherbus[weatherbus["Visibility (km)"].isna()].index)
weatherbus=weatherbus.drop(weatherbus[weatherbus["Wind Spd (km/h)"].isna()].index)
weatherbus=weatherbus.drop(weatherbus[weatherbus["Rel Hum (%)"].isna()].index)
weatherbus=weatherbus.drop(weatherbus[weatherbus["Wind Dir (10s deg)"].isna()].index)
weatherbus.drop("Vehicle", axis=1, inplace=True)
weatherbus.drop("Direction", axis=1, inplace=True)
weatherbus=weatherbus.drop(weatherbus[weatherbus["Delay"].isna()].index)
weatherbus=weatherbus.drop(weatherbus[weatherbus["Gap"].isna()].index)
weatherbus=weatherbus.drop(weatherbus[weatherbus["Incident"].isna()].index)
weatherbus = weatherbus.fillna('na')

weatherbus["Rain"]=weatherbus["Weather"].apply(lambda row: 1 if 'Rain' in row else 0)
weatherbus["Cloudy"]=weatherbus["Weather"].apply(lambda row: 1 if 'Cloudy' in row else 0)
weatherbus["Snow"]=weatherbus["Weather"].apply(lambda row: 1 if 'Snow' in row else 0)
weatherbus["Fog"]=weatherbus["Weather"].apply(lambda row: 1 if 'Fog' in row else 0)
weatherbus["Freezing"]=weatherbus["Weather"].apply(lambda row: 1 if 'Freezing' in row else 0)
weatherbus["Drizzle"]=weatherbus["Weather"].apply(lambda row: 1 if 'Drizzle' in row else 0)
weatherbus["Thunderstorms"]=weatherbus["Weather"].apply(lambda row: 1 if 'Thunderstorms' in row else 0)
weatherbus.drop("Weather", axis=1, inplace=True)


temp = ((weatherbus.Day[1:].values-weatherbus.Day[:-1].values)*24+weatherbus.Hour[1:].values-weatherbus.Hour[:-1].values)*60+weatherbus.Minute[1:].values-weatherbus.Minute[:-1].values
temp = np.insert(temp,0,0)
weatherbus["deltaT"]=temp
weatherbus["deltaT"]=weatherbus["deltaT"].apply(lambda row: 0 if row < 0 else row)
prediction_var = 'Incident'
exit_code = {}
for a, i in enumerate(sorted(set(weatherbus[prediction_var]))):
    exit_code[i]=a
weatherbus[prediction_var]=weatherbus[prediction_var].map(exit_code)



### Separating data

In [7]:
trainingData = weathersubway[:int(0.6*len(weathersubway))]
validationData = weathersubway[int(0.6*len(weathersubway)):int(0.8*len(weathersubway))]
testData = weathersubway[int(0.8*len(weathersubway)):]

In [20]:
X = trainingData.copy()
yDelay = X["Min Delay"]
yGap = X["Min Gap"]
yCode = X["Code"]
X.drop("Code", axis=1, inplace=True)
X.drop("Min Delay", axis=1, inplace=True)
X.drop("Min Gap", axis=1, inplace=True)
X.reset_index(inplace=True)
X.drop("index", axis=1, inplace=True)

If some columns are still qualitative, transfer. (Subway)

In [6]:
exit_code = {}
for a, i in enumerate(sorted(set(weathersubway["Bound"]))):
    exit_code[i]=a
weathersubway["Bound"]=weathersubway["Bound"].map(exit_code)
exit_code = {}
for a, i in enumerate(sorted(set(weathersubway["Line"]))):
    exit_code[i]=a
weathersubway["Line"]=weathersubway["Line"].map(exit_code)

## Modelling

### Modelling with a Random Forest (memoryless)

#### With pipeline

In [115]:
rf_classifier = RandomForestClassifier(min_samples_leaf=50,n_estimators=100,oob_score=True,n_jobs=-1,random_state=0,max_features='auto')
pipe = make_pipeline(col_trans, rf_classifier)
pipe.fit(X, yCode)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Route', 'Direction'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(min_samples_leaf=50, n_jobs=-1,
                                        oob_score=True, random_state=0))])

#### Without pipeline

In [31]:
rf_classifier = RandomForestClassifier(min_samples_leaf=5,n_estimators=100,oob_score=True,n_jobs=-1,random_state=0,max_features='auto')
rf_classifier.fit(X, yCode)

RandomForestClassifier(min_samples_leaf=5, n_jobs=-1, oob_score=True,
                       random_state=0)

### Modelling with an SVM (memoryless)

In [98]:
svm_classifier = SVC()
pipe_svm = make_pipeline(col_trans, svm_classifier)
pipe_svm.fit(trainingData, trainingLabels)
y_pred_validate_2 = pipe_svm.predict(validationData)

### Modelling with a fully connected neural_network (memoryless)

In [105]:
mlp_classifier = MLPClassifier(hidden_layer_sizes=(10,10,10,))
pipe_mlp = make_pipeline(col_trans, mlp_classifier)
pipe_mlp.fit(trainingData, trainingLabels)
y_pred_validate_3 = pipe_mlp.predict(validationData)

### Modelling an LSTM

Initial model.

In [35]:
scaler = MinMaxScaler(feature_range = (0, 1))

bus_scaled = scaler.fit_transform(trainingData)

features_set = []
labels = []
for i in range(60, len(bus_scaled)):
    features_set.append(bus_scaled[i-60:i])
    labels.append(bus_scaled[i, 3]) # 2: Incident 3: Delay 4: Gap
features_set, labels = np.array(features_set), np.array(labels)
features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], trainingData.shape[1]))

#### Change the LSTM size depending on 
layer_size = 20

model = Sequential()
model.add(LSTM(units=layer_size, return_sequences=True, input_shape=(features_set.shape[1], trainingData.shape[1])))
model.add(Dropout(0.2))

model.add(LSTM(units=layer_size, return_sequences=True)) # 4 LSTM layers, and 0.2 dropout.
model.add(Dropout(0.2))

model.add(LSTM(units=layer_size, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=layer_size))
model.add(Dropout(0.2))

model.add(Dense(units = 1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

Training.

In [36]:
model.fit(features_set, labels, epochs = 20, batch_size = 32)

Epoch 1/20
2748/2748 [==============================] - 96s 34ms/step - loss: 1.2229e-04
Epoch 2/20
2748/2748 [==============================] - 93s 34ms/step - loss: 9.7276e-05
Epoch 3/20
2748/2748 [==============================] - 94s 34ms/step - loss: 9.5954e-05
Epoch 4/20
2748/2748 [==============================] - 98s 35ms/step - loss: 9.5116e-05
Epoch 5/20
2748/2748 [==============================] - 97s 35ms/step - loss: 9.4976e-05
Epoch 6/20
2748/2748 [==============================] - 95s 35ms/step - loss: 9.5108e-05
Epoch 7/20
2748/2748 [==============================] - 95s 35ms/step - loss: 9.4993e-05
Epoch 8/20
2748/2748 [==============================] - 95s 35ms/step - loss: 9.4994e-05
Epoch 9/20
2748/2748 [==============================] - 95s 35ms/step - loss: 9.5066e-05
Epoch 10/20
2748/2748 [==============================] - 95s 35ms/step - loss: 9.4973e-05
Epoch 11/20
2748/2748 [==============================] - 95s 35ms/step - loss: 9.5001e-05
Epoch 12/20
2748/27

## Validation

#### With pipeline, memoryless models

In [117]:
y_pred_validate = pipe.predict(X)
f1_score(y_pred_validate, yCode.values, average = "weighted")

ValueError: Found unknown categories [898.0, 900.0, 389.0, 106117.0, 903.0, 137.0, 138.0, 268.0, 524.0, 913.0, 146.0, 789.0, 151.0, 666.0, 924.0, 925.0, 158.0, 927.0, 929.0, 166.0, 935.0, 936.0, 937.0, 938.0, 939.0, 557.0, 813.0, 941.0, 176.0, 944.0, 945.0, 183.0, 952.0, 954.0, 187.0, 700.0, 958.0, 960.0, 963.0, 714.0, 716.0, 719.0, 720.0, 847.0, 978.0, 212.0, 468.0, 600.0, 984.0, 474.0, 219.0, 603.0, 604.0, 606.0, 985.0, 608.0, 737.0, 986.0, 611.0, 995.0, 359.0, 872.0, 876.0, 241.0, 369.0, 758.0] in column 0 during transform

#### Without pipeline, memoryless models

In [142]:
y_pred_validate = rf_classifier.predict(X)
confusion_matrix(y_pred_validate, yCode.values)

array([[   27,     0,    17,     1,    17,     8,     2,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0],
       [  746,    57, 12127,   313,   294,  1339,   691,     0],
       [    1,     0,     0,     2,     0,     3,     1,     0],
       [ 1113,   141,   456,   580, 10459,  3988,  2105,     0],
       [ 4692,  1874,  3072,  8955,  6087, 24070,  9986,     0],
       [  261,   119,   901,   744,  1256,  1893,  3468,     1],
       [    0,     0,     0,     0,     0,     0,     0,     0]],
      dtype=int64)

#### LSTM

Scale test data.

In [37]:
bus_scaled = scaler.transform(testData)
features_set = []
labels = []
for i in range(60, len(bus_scaled)):
    features_set.append(bus_scaled[i-60:i])
    labels.append(bus_scaled[i, 3]) # 2: Incident 3: Delay 4: Gap
features_set, labels = np.array(features_set), np.array(labels)
features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], validationData.shape[1]))

Prediction.

In [38]:
predicted_labels = model.predict(features_set)

Error.

In [39]:
from sklearn.metrics import mean_squared_error
mean_squared_error(labels, predicted_labels)

7.220477553585571e-05

For all models:

### Save model.

In [40]:
model.save('lstm_subway_gap.h5')

# Annex

Processing coordinates, subway dataset. (trial-and-error)

In [10]:
from geopy.geocoders import Nominatim
import re
def getCoordinates(location):
    loc = Nominatim(user_agent="GetLoc")
    location=re.split('\(',location)[0]
    location=re.split('TO',location)[0]
    location=re.split('-',location)[0]
    getLoc = loc.geocode("{}, Toronto".format(location))
    if (getLoc is None):
        location=' '.join(re.split(' ',location.strip())[:-1]) #maybe a word too much
        getLoc = loc.geocode("{}, Toronto".format(location))
        if (getLoc is None):
            return (None, None)
    return (getLoc.latitude,getLoc.longitude)

coordinateDictionary= {}
for location in set(weathersubway["Station"]):
    coordinateDictionary[location] = getCoordinates(location)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=HIGHWAY+407%2C+Toronto&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

### Saving dataframes

Invert the label for clean interpretable output (any prediction_var)

In [ ]:
weathersubway[prediction_var]=weathersubway[prediction_var].map(dict(map(reversed, exit_code.items())))

Save to csv.

In [104]:
weatherbus.to_csv("{}\\busdata.csv".format(os.getcwd()))
weathersubway.to_csv("{}\\subwaydata.csv".format(os.getcwd()))
dfeast.to_csv("{}\\eastdata.csv".format(os.getcwd()))
dfcenter.to_csv("{}\\buscenter.csv".format(os.getcwd()))

Processed coordinates, subway dataset.

In [3]:
coords = {'DAVISVILLE CARHOUSE': (43.6959713, -79.39732267921579),
 'KENNEDY BD STATION (PL': (43.732508, -79.2637953),
 'LANSDOWNE STATION': (43.6593866, -79.4432055),
 'ST CLAIR WEST TO VMC S': (43.6849986, -79.4157296),
 'WOODBINE STATION (APPR': (43.6863861, -79.31271156664201),
 'ST GEORGE TO FINCH WES': (43.6684465, -79.398844),
 'DUNDAS STATION - WEST': (43.6570301, -79.4529728),
 'CHRISTIE STATION (LEAV': (43.6640809, -79.4185289),
 'GLENAYR EE': (43.6962511, -79.4197343),
 'KENNEDY TO MCCOWAN - L': (43.732508, -79.2637953),
 'DUPONT STATION APPROAC': (43.6747046, -79.406983),
 'CASTLE FRANK TO SHERBO': (43.673509, -79.36903055286996),
 'UNION, KIPLING, ROYAL': (43.6456424, -79.3802085),
 'OLD MILL TO ISLINGTON': (43.649826, -79.4943338),
 'BLOOR/DANFORTH LINE': (43.6443557, -79.527766),
 'UNION, DOWNSVIEW PARK,': (43.6456424, -79.3802085),
 'BAY STATION': (43.6701851, -79.3902694),
 "QUEEN'S PARK TO ST CLA": (43.66450665, -79.39235972390199),
 'DUNCAN SHOPS': (43.6534817, -79.3839347),
 'KING TO EGLINTON STATO': (43.6489494, -79.3777538),
 'EGLINTON POCKET': (43.6534817, -79.3839347),
 'YONGE AUTO ENTRANCE': (43.7295792, -79.4026533),
 'MCCOWAN PARKING LOT': (43.7749091, -79.2515777),
 'MCCOWAN TO KENNEDY STA': (43.7749091, -79.2515777),
 'LINE 1, LINE 2 AND LIN': (None, None),
 'DAVISVILLE BUID-UP': (43.6534817, -79.3839347),
 'VMC STATION TO SHEPPAR': (43.6534817, -79.3839347),
 'KIPLING STATION (APPRO': (43.63753245, -79.53560410315569),
 'CHURCH EMERGENCY EXIT': (43.684529850000004, -79.41389249904516),
 'RUSSELL HILL EMERGENCY': (43.6822234, -79.4109543),
 'OSSINGTON STAITON': (43.6534817, -79.3839347),
 'OAKVALE SUBSTATION': (43.6534817, -79.3839347),
 'YONGE SHEPPARD STATION': (43.7614518, -79.4109148),
 'WILSON STATION': (43.7347436, -79.4508641),
 'FINCH TO VAUGHAN METRO': (43.7812974, -79.4158993),
 'KEELE TO OSSINGTON': (43.655569, -79.4595795),
 'YORK MILLS TO LAWRENCE': (43.7440391, -79.406657),
 'SHEPPARD WEST MIGRATI': (43.733027, -79.5316661),
 'SHEPPARD- YONGE STATIO': (43.733027, -79.5316661),
 'DAVISVILLE YARD (APPRO': (43.6959713, -79.39732267921579),
 'QUEEN TO KING STATIONS': (43.6529083, -79.3794575),
 'ST GEORGE TO BROADVIEW': (43.6684465, -79.398844),
 'YONGE-UNIVERSITY-SPADI': (43.6541457, -79.38079276646786),
 'LESLIE STATION(APPROAC': (43.7705457, -79.3678254),
 'WILSON NORTH HOSTLER': (43.7344762, -79.4500619),
 'JANE STATION (ENTERING': (43.6499177, -79.4838463),
 'WILSON STATION TO YORK': (43.7347436, -79.4508641),
 'FENMAR AND WESTON': (43.7649732, -79.5621868),
 'KING TO S/O ST ANDREW': (43.6489494, -79.3777538),
 'SHEPPARD TO EGLINTON S': (43.733027, -79.5316661),
 'FINCH WEST TO LAWRENCE': (43.7646446, -79.4913677),
 'YUS/BD/SHEPPARD SUBWAY': (43.6534817, -79.3839347),
 'EGLINTON TO BLOOR STAT': (43.6534817, -79.3839347),
 'DUNDAS WEST STATION -': (43.6569564, -79.4538893),
 'YORK UNIVERSTITY, DOWN': (43.6896191, -79.479188),
 'ROSEDALE STATION': (43.676735050000005, -79.3886162498014),
 'DUNDA WEST STATION': (None, None),
 'PRINCE EDWARD SUBSTATI': (46.4524682, -63.3799629),
 'ST CLAIR STATION': (43.6879575, -79.3918194),
 'EGLINTON TO SHEPPARD': (43.6534817, -79.3839347),
 'LEAVING SHEPPARD/YONGE': (43.6534817, -79.3839347),
 'MIDLAND STATION': (43.7704537, -79.271982),
 'ST GEORGE TO GREENWOOD': (43.6684465, -79.398844),
 'DUPONT MIGRATION POINT': (43.6747046, -79.406983),
 'KEELE STATION TO KIPLI': (43.6554366, -79.4596039),
 'OSGOODE STATION': (43.6508016, -79.3865409),
 'LINE 3 SCARBOROUGH SRT': (43.6093021, -79.5493908),
 'SHEPPARD WEST TO FINCH': (43.7492988, -79.462248),
 'HIGHWAY 407 STATION': (38.2975037, 140.9254559),
 'BESSARION STATION': (43.7690604, -79.37588370821047),
 'YU & BD': (-1.0525169, -46.76383392547042),
 'YONGE BD STATION': (43.6541457, -79.38079276646786),
 'SPADINA YUS STATION (L': (43.6685404, -79.4047039),
 'UNION STATION (ST ANDR': (43.6447302, -79.38005624881814),
 'ELLESMERE STATION DEPA': (43.7668074, -79.2762856),
 'UNION TO KING STATION': (43.6456424, -79.3802085),
 'STC TO MCCOWAN STATION': (43.7754419, -79.2577414),
 'OSSIGNTON STATION': (43.6534817, -79.3839347),
 'KING COMMERCE': (43.6486783, -79.3801578),
 'W/O CASTLE FRANK TO GR': (-33.5078555, -70.7315805),
 'YONGE AND QUEEN': (43.6541457, -79.38079276646786),
 'SCARBORUGH RAPID TRANS': (43.7669017, -79.2774783),
 'RUNNYMEDE STATION': (43.6517026, -79.4759978),
 'KIPLING STATION TO COX': (43.63753245, -79.53560410315569),
 'COXWELL STATION': (43.6841534, -79.3231599),
 'WELLBECK EMERGENCY': (43.6534817, -79.3839347),
 'COXWELL STATION (LEAVI': (43.6841534, -79.3231599),
 'ST CLAIR WEST CENTRE T': (43.6849986, -79.4157296),
 'BROADVIEW TO ST. GEORG': (43.6769999, -79.35823476332092),
 'KING TO UNION': (43.6489494, -79.3777538),
 'BLOOR VIADUCT': (43.6721285, -79.3705786),
 'KENNEDY BD STATION (LE': (43.732508, -79.2637953),
 "QUEEN'S QUAY STATION": (43.66450665, -79.39235972390199),
 'YONGE / UNIVERSITY - B': (43.6707855, -79.3856867),
 'YONGE UNIVERSITY': (43.6707855, -79.3856867),
 'ST GEORGE STATION TO W': (43.6686414, -79.39801880470816),
 'EGLINTON STATION TO SH': (43.6534817, -79.3839347),
 'ST. CLAIR WEST TO DUPO': (43.6849986, -79.4157296),
 'YONGE STATION (EXITING': (43.6491143, -79.3783718),
 'UNION (TO ST ANDREW)': (43.6456424, -79.3802085),
 'KENNEDY TO MCCOWAN': (43.732508, -79.2637953),
 'KIPLING TO ISLINGTON S': (43.6376161, -79.5355081),
 'KILPING STATION': (43.6534817, -79.3839347),
 'DOWNSVIEWSTATION': (43.6534817, -79.3839347),
 'KING TO ST GEORGE STAT': (43.6489494, -79.3777538),
 'YONGE/UNIVERSITY BLOOR': (43.6707855, -79.3856867),
 'DAVISVILLE TO N/O BLOO': (43.697936, -79.3972908),
 'BLOOR-DANFORTH LINE': (43.6577734, -79.4507397),
 'YONGE/UNIVERSITY SUBWA': (43.6534817, -79.3839347),
 'BLOOR STATION (STATION': (43.6577734, -79.4507397),
 'CHESTER TO CASTLEFRANK': (43.6782277, -79.3525143),
 'WILSON GARAGE': (43.6534817, -79.3839347),
 'YUS/BD': (43.6534817, -79.3839347),
 'BLOOR STATION': (43.6577734, -79.4507397),
 'GREENWOOD COMPLEX': (43.67888915, -79.33218372970026),
 'ENTERING LESLIE STATIO': (43.7703702, -79.3676557),
 'OSGOODE': (43.6508016, -79.3865409),
 'SOUTH OF LAWRENCE SRT': (-33.013441, 151.5942043),
 'UNION TO FINCH': (43.6456424, -79.3802085),
 'SHEPPARD-YONGE (TAIL T': (43.733027, -79.5316661),
 'ROYAL YORK STATION (LE': (43.6481827, -79.5112961),
 'YONGE UNIVERSTIY SUBWA': (43.6541457, -79.38079276646786),
 'SPADINA YUS STATION': (43.6685404, -79.4047039),
 'BLOOR DANFORTH SUBWAY': (43.6387762, -79.5340283),
 'KIPLING STATION TO KEN': (43.63753245, -79.53560410315569),
 'ST CALIR STATION': (43.65841965, -79.38313842626155),
 'HARVEY SHOP': (43.6534817, -79.3839347),
 'LINE 1 AND LINE 2 AND': (43.6534817, -79.3839347),
 'GREENWOOD SHOPS': (43.6534817, -79.3839347),
 'BLOOR INTERLOCKING': (43.6534817, -79.3839347),
 'KENNEDY TO KIPLING STA': (43.732508, -79.2637953),
 'SHERBOURNE AND BLOOR': (43.6722828, -79.376394),
 'ST CLAIR TO UNION STAT': (43.6879512, -79.3936947),
 'N/O QUEEN TO MARKDALE': (50.8272552, -0.1265822),
 'KENNEDY SRT STATION (L': (43.732508, -79.2637953),
 'YONGE/UNIVERSITY - BLO': (43.6707855, -79.3856867),
 'KING TO EGLINTON STATI': (43.6489494, -79.3777538),
 'SYSTEM WIDE': (43.6534817, -79.3839347),
 'DAVISVILLE YARD - TUNN': (43.6959713, -79.39732267921579),
 'ENGLINTON TO SHEPPARD': (43.6534817, -79.3839347),
 'LAWRENCE WEST STATION': (43.7154738, -79.4445163),
 'LEAVING SHEPPARD-YONGE': (43.6534817, -79.3839347),
 'EGLINTON TO FINCH': (43.6534817, -79.3839347),
 'COLLEGE TO KING': (43.6606617, -79.3827952),
 'YORKDALE TO DUPONT STA': (43.7246418, -79.4475031),
 'GREENWOOD YARD': (43.6768324, -79.33205534218284),
 'SCARBOROUGH CTR STATIO': (43.7729744, -79.2576479),
 'BROADVIEW TO YONGE': (43.6769999, -79.35823476332092),
 'FINCH TO EGLINTON STAT': (43.7812974, -79.4158993),
 'DOWNSVIEW STATION PLAT': (43.7492988, -79.462248),
 'BLOOR VIA DUCT': (43.6577734, -79.4507397),
 'BESSARIAN STATION': (43.6534817, -79.3839347),
 'UNION STATION TO FINCH': (43.6447302, -79.38005624881814),
 'DAVISVILLE BUILDD UP': (43.697936, -79.3972908),
 'HIGH PARK TO KEELE STA': (43.6538668, -79.4668644),
 'KEELE STATION (CROSSIN': (43.6554366, -79.4596039),
 'WILSON SOUTH HOSTLER': (43.7344762, -79.4500619),
 'KING STATION - COMMERC': (43.649114299999994, -79.37837180000002),
 'WARDEN STATION (LEAVIN': (43.7109797, -79.2797044),
 'N/O WILSON TO S/O SHEP': (43.7398265, -79.418975),
 'KENNEDY STATION': (43.7323896, -79.2637081),
 'KENNEDY BD STATION (AP': (43.732508, -79.2637953),
 'BLOOR TO ST ANDREW STA': (43.6577734, -79.4507397),
 'MCCOWAN CAR HOUSE': (43.7749091, -79.2515777),
 'FINCH TO QUEEN STATION': (43.7812974, -79.4158993),
 'GLENCAIRN STATION (APP': (43.7087117, -79.4406853),
 'PIONEER VILLAGE STATIO': (43.683411, -79.3010791),
 'ST CLAIR WEST STATION': (43.6841761, -79.4155252),
 'KING STATION': (43.649114299999994, -79.37837180000002),
 'UNION STATION TO ST AN': (43.6447302, -79.38005624881814),
 'LESLIE STATION (LEAVIN': (43.7705457, -79.3678254),
 'TORONTO TRANSIT COMMIS': (43.6534817, -79.3839347),
 'JANE STATION': (43.6499177, -79.4838463),
 'WELLESLEY STATION (APP': (43.665403, -79.3836001),
 'FINCH STATION (LEAVING': (43.7827501, -79.41488829825582),
 'YUS/BD/SRT': (43.6534817, -79.3839347),
 'OSSINGTON STATION': (43.6534817, -79.3839347),
 "QUEEN'S QUAY ELEVATOR": (43.66450665, -79.39235972390199),
 'ST CLAIR STATION PLEAS': (43.6879512, -79.3936947),
 'WOODBINE STATION (LEAV': (43.6863861, -79.31271156664201),
 'LINE 1': (43.7369134, -79.6302733),
 'UNION TO ST GEORGE': (43.6456424, -79.3802085),
 'CHESTER TO WARDEN': (43.6782277, -79.3525143),
 'YONGE AND STEELES': (43.6541457, -79.38079276646786),
 'OLD MILL STATION': (43.6501618, -79.495091),
 'ROYAL YORK TO ISLINGTO': (43.6481827, -79.5112961),
 'LAWRENCE AND YONGE': (43.7263483, -79.4024743),
 'PAPE STATION': (43.67996425, -79.34474097964232),
 'ISLINGTON TO PRINCE ED': (43.6534817, -79.3839347),
 'ST PATRICK STATION': (43.6548199, -79.3882736),
 'YONGE-UNIVERSITY / BLO': (43.6541457, -79.38079276646786),
 'WARDEN STATION': (43.7109797, -79.2797044),
 'YONGE-UNIVERSITY & BLO': (43.6541457, -79.38079276646786),
 'KING STATION TO ST CLA': (43.649114299999994, -79.37837180000002),
 'KENNEDY BD STATION': (43.732508, -79.2637953),
 'BLOOR YONGE LINES': (43.6577734, -79.4507397),
 'DONLANDS STATION': (43.6809856, -79.33781360458943),
 'YORKDALE TO FINCH STAT': (43.7246418, -79.4475031),
 'ST ANDREW STATION (LEA': (43.6475462, -79.3854279),
 'KIPLING - ISLIINGTON S': (43.6376161, -79.5355081),
 'FINCH WEST - SHEPPARD': (43.7646446, -79.4913677),
 'YONGE BD STATION BOOTH': (43.7347737, -79.4071414),
 'KIPLING TO ROYAL YORK': (43.6376161, -79.5355081),
 'ST GEORGE TO WOODBINE': (43.6684465, -79.398844),
 'UNION STATION (KING)': (43.6447302, -79.38005624881814),
 'CHESTER CENTRE TRACK': (43.6782277, -79.3525143),
 'ST CLAIR E OF BATHURST': (43.6879512, -79.3936947),
 'YONGE-UNIVERSITY LINE': (43.6541457, -79.38079276646786),
 'GREENWOOD SHOP': (43.6534817, -79.3839347),
 'DUFFERIN STATION': (43.6602019, -79.4357191),
 'KIPLING STATION': (43.63753245, -79.53560410315569),
 'DOWNSVIEW PARK TO VMC': (43.7536239, -79.47894281136364),
 'RONCEVALLES DIVISION': (43.6534817, -79.3839347),
 'MUSEUM STATION TO OSGO': (43.667097, -79.3936065),
 'GLENCAIRN STATION': (43.7087117, -79.4406853),
 'ROYAL YORK STATION (EN': (43.6481827, -79.5112961),
 'ST GEORGE/BEDFORD': (43.6686081, -79.398161),
 'FINCH STATION (PLATFOR': (43.7827501, -79.41488829825582),
 'UNION TO BLOOR STATION': (43.6456424, -79.3802085),
 'WILSON STRUCTURE BUILD': (43.7344762, -79.4500619),
 'SUBWAY OPERATIONS BLDG': (43.674406399999995, -79.4162633234271),
 'OSSINGTON AND DUFFERIN': (43.6534817, -79.3839347),
 'GREENWOOD TRACK AND ST': (43.6667687, -79.3134453),
 'UNION STATION (TO ST A': (43.6447302, -79.38005624881814),
 'MCCOWAN STATION (DEPAR': (43.7749091, -79.2515777),
 'GLENCAIRN': (43.7087117, -79.4406853),
 'MCCOWAN STATION (ENTER': (43.7749091, -79.2515777),
 'KENNEDY SRT STATION [A': (43.73239336, -79.26383039),
 'ST CLAIR STATION - UPP': (43.6879575, -79.3918194),
 'LAWRENCE TO BLOOR STAT': (43.7263483, -79.4024743),
 'ELLIS BRIDGE': (43.6534817, -79.3839347),
 'LAWRENCE SUB STATION': (43.7263483, -79.4024743),
 'LANSDOWN STATION': (43.6534817, -79.3839347),
 'COXWELL STATION (EXITI': (43.6841534, -79.3231599),
 'KIPLING STATION (EXITI': (43.63753245, -79.53560410315569),
 'UNION STATION (APPROAC': (43.6447302, -79.38005624881814),
 'CHESTER STATION (LEAVI': (43.6782277, -79.3525143),
 'YORK MILLS STATION': (43.745179, -79.4061287),
 'PRINCE EDWARD VIADUCT': (43.675198300000005, -79.36402949329647),
 'DAVISVILLE YARD': (43.6959713, -79.39732267921579),
 'BEDFORD SUBSTATION': (43.6534817, -79.3839347),
 'ST GEORGE YUS STATION': (43.6684465, -79.398844),
 'BLOOR AND ABERFOYLE CR': (43.668164, -79.39376195),
 'LEAVING YONGE-SHEPPARD': (43.6534817, -79.3839347),
 'WELLSLEY STATION': (43.6534817, -79.3839347),
 'YORKDALE STATION': (43.7246418, -79.4475031),
 'WILSON SUBWAY YARD': (43.7344762, -79.4500619),
 'WILSON DIVISION TRAILE': (43.7344762, -79.4500619),
 'SPADINA YU STATION': (43.6685404, -79.4047039),
 'SYSTEM - WIDE IN THE O': (43.7290756, -79.59788584813774),
 'KEELE STATION (EXITING': (43.6554366, -79.4596039),
 'DUNDAS STATION - W/S': (43.6570301, -79.4529728),
 'LEAVING LAWRENCE EAST': (43.6569827, -79.4527908),
 'BLOOR STATON': (43.6721792, -79.376653),
 'QUEEN ALBERT': (43.6534817, -79.3839347),
 'KENNEDY TO LAWRENCE EA': (43.732508, -79.2637953),
 'EGLINTON - FINCH STATI': (43.6534817, -79.3839347),
 'KIPLING STATION TO ISL': (43.63753245, -79.53560410315569),
 'SCARBOROUGH SRT LINE': (43.7729744, -79.2576479),
 'WILSON HOSLTER': (43.6534817, -79.3839347),
 'YONGE / UNIVERSITY - L': (43.6707855, -79.3856867),
 'UNION STATION - SALES': (43.6447302, -79.38005624881814),
 'LAWRENCE TO EGLINTON S': (43.7263483, -79.4024743),
 'FINCH STATION TO EGLIN': (43.7827501, -79.41488829825582),
 'YORK MILLS STATION (LE': (43.745179, -79.4061287),
 'FINCH TO ROSEDALE': (43.7812974, -79.4158993),
 'DOWNSVIEW STATION': (43.74925275, -79.46222522403818),
 'SEHPPARD STATION': (43.6534817, -79.3839347),
 'FINCH STATION BOOTH 2': (43.7827501, -79.41488829825582),
 'YONGE SHP STATION': (43.6541457, -79.38079276646786),
 'SPADINA STATION': (43.6672349, -79.4036863),
 'FINCH WEST TO WILSON': (43.7646446, -79.4913677),
 'KENNEDY PLATFORM 1': (43.7658499, -79.2810728),
 'UNION STATION TO ST. A': (43.6447302, -79.38005624881814),
 'GREENWOOD AND DANFORTH': (43.68253285, -79.33024260293422),
 'SCARB CENTRE STATION': (43.77433626, -79.2583270),
 'CHESTER STATION (ENTER': (43.6782277, -79.3525143),
 'WILSON CARHOUSE': (43.6534817, -79.3839347),
 'WILSON SUB STATION': (43.7344762, -79.4500619),
 'WILSON HOSTLER SOUTHBO': (43.7344762, -79.4500619),
 'SCARBOROUGH CENTRE STA': (43.774427, -79.25778877539925),
 'LEAVING YONGE/SHEPPARD': (43.6534817, -79.3839347),
 'KENNEDY SRT STATION TO': (43.732508, -79.2637953),
 'ROSEDALE STATION (APP': (43.676735050000005, -79.3886162498014),
 'EGLINTON STATION (APPR': (43.6534817, -79.3839347),
 'UNION STATION (ENTERIN': (43.6447302, -79.38005624881814),
 'DUFFERIN TO KIPLING ST': (43.6602019, -79.4357191),
 'SHEPPARD-YONGE TAIL': (43.733027, -79.5316661),
 'ST CLAIR WEST TO ST AN': (43.6849986, -79.4157296),
 'KENNEDY': (43.732508, -79.2637953),
 'VMC TO SHEPPARD WEST': (43.6534817, -79.3839347),
 'ST GEORGE YU STATION': (43.6684465, -79.398844),
 'WILSON TO EGLINTON': (43.7344762, -79.4500619),
 'ISLINGTON STATION ( AP': (43.6534817, -79.3839347),
 'BAYVIEW STATION': (43.7667987, -79.3873787),
 'MUSEUM TO FINCH STATIO': (43.6668462, -79.3933506),
 'BLOOR TO UNION': (43.6577734, -79.4507397),
 'GLENCAIRN TO ST GEORGE': (43.7087117, -79.4406853),
 'KING STATION TO UNION': (43.649114299999994, -79.37837180000002),
 'DUPLEX AVENUE': (43.7213551, -79.4044198),
 'YONGE BD STATION (LEAV': (43.6541457, -79.38079276646786),
 'YONGE/UNIVERISITY-BLOO': (43.6534817, -79.3839347),
 'KENNEDY BD STATION (EN': (43.732508, -79.2637953),
 'EGLINTON TO LAWRENCE': (43.6534817, -79.3839347),
 'MCBRIEN BUILDING': (43.6534817, -79.3839347),
 'BLOOR DANFORTH LINE': (43.6443557, -79.527766),
 'SHEPPARD-YONGE STATION': (43.733027, -79.5316661),
 'JANE STATION TO OLD MI': (43.6499177, -79.4838463),
 'FINCH STATION TO DUPON': (43.7827501, -79.41488829825582),
 'MCBRIEN BUIDING': (43.6534817, -79.3839347),
 'MCCOWAN STATION TO KEN': (43.7749091, -79.2515777),
 'MAIN STREET STATION': (43.6891437, -79.3016858),
 'EGLINTON WEST TO ST CL': (43.6534817, -79.3839347),
 'SCARBOROUGH RT LINE': (43.7729744, -79.2576479),
 'YONGE - UNIVERSITY LIN': (43.6541457, -79.38079276646786),
 'SHERBOURNE STATION': (43.6721875, -79.3764921),
 'ATC TERRIROTY': (43.6534817, -79.3839347),
 'EGLINTON STATION (ENTE': (43.6534817, -79.3839347),
 'MCCOWAN YARD - 3 TRACK': (43.7759581, -79.24528165482738),
 'DUNCAN SHOP': (43.6534817, -79.3839347),
 'INGLIS BUILDING': (43.7878432, -79.4536805),
 'COXWELL STATION (ENTER': (43.6841534, -79.3231599),
 'WILSON STATION TO VMC': (43.7347436, -79.4508641),
 'FULL LINE': (43.6534817, -79.3839347),
 'ST ANDREW TO BLOOR STA': (43.6477917, -79.3848711),
 'EGLINTON TO LAWRENCE S': (43.6534817, -79.3839347),
 'UNION, MAIN, KENNEDY S': (43.7324243, -79.2638733),
 'SCARBOROUGH RAOID TRAN': (43.7729744, -79.2576479),
 'E13': (25.7914878, -100.273335),
 'UNION TO ST ANDREW': (43.6456424, -79.3802085),
 'WILSON STATION- HOSTLE': (43.7347436, -79.4508641),
 'SHEPPARD TAIL TRACK #2': (43.7861939, -79.2739508),
 'GLENACAIRN STATION (AP': (43.6534817, -79.3839347),
 'FINCH TO ST CLAIR STAT': (43.7812974, -79.4158993),
 'BROADVIEW STATION TO W': (43.6770471, -79.35839530606879),
 'UNION STATION (TO KING': (43.6447302, -79.38005624881814),
 'EGLINTON WEST STATION': (43.6534817, -79.3839347),
 'ST. CLAIR WEST TO KING': (43.6849986, -79.4157296),
 'ST CLAIR TO SHEPPARD S': (43.6879512, -79.3936947),
 'ISLINGTON STATION (LEA': (43.6534817, -79.3839347),
 'ISLINGTON CENTRE TRACK': (43.6534817, -79.3839347),
 'WELLESLEY TO COLLEGE': (43.6655242, -79.3838011),
 'YONGE UNIVESITY AND BL': (43.7307890, -79.4491296),
 'VMC STATION': (43.6534817, -79.3839347),
 'NORTH YORK CTR STATION': (43.7686787, -79.4126298),
 'ST ANDREW TO BLOOR ST': (43.6477917, -79.3848711),
 'KIPLING STATION TO JAN': (43.63753245, -79.53560410315569),
 'SUMMERHILL TO BLOOR ST': (43.6816776, -79.3905037),
 'YONGE/UNIVERSITY/SPADI': (43.6534817, -79.3839347),
 'ZONE 1': (14.7256346, 121.0084849),
 'ST CLAIR TO UNION': (43.6879512, -79.3936947),
 'DUPONT STATION (EXITIN': (43.6747046, -79.406983),
 'YORK MILLS STATION [': (43.745179, -79.4061287),
 'FINCH STATION': (43.7827501, -79.41488829825582),
 'ST GEORGE BD STATION': (43.6684465, -79.398844),
 'BLOOR TO COLLEGE': (43.6577734, -79.4507397),
 'KEELE TO DUNDAS WEST': (43.655569, -79.4595795),
 'ST CLAIR WEST': (43.6849986, -79.4157296),
 'KENNEDY TO KIPLING': (43.732508, -79.2637953),
 'CHRISTIE STATION (APPR': (43.6640809, -79.4185289),
 'WARDEN STATION TO KENN': (43.7109797, -79.2797044),
 'DONLANDS STATION (ENTE': (43.6809856, -79.33781360458943),
 'LINE 3 - SCARBOROUGH R': (43.6093021, -79.5493908),
 'SHEPPARD WEST TO WILSO': (43.7492988, -79.462248),
 'BRUNSWICK AND BLOOR': (43.665564, -79.403479),
 'YORK MILLS TO ST CLAIR': (43.7440391, -79.406657),
 'PIONEER VILLAGE TO VAU': (43.7749816, -79.51888207794511),
 'MUSEUM STATION (APPROA': (43.667097, -79.3936065),
 'KEELE STATION (APPROAC': (43.6554366, -79.4596039),
 'VICTORIA PARK TO KENNE': (43.66692875, -79.39195217788848),
 'GREENWOOD CARHOUSE': (43.6534817, -79.3839347),
 'WILSON TRAINING BUILDI': (43.7344762, -79.4500619),
 'GREENWOOD CAR HOUSE': (43.68253285, -79.33024260293422),
 'SCARB CTR STATION': (None, None),
 'DAVISVILLE BUILD-UP': (43.6534817, -79.3839347),
 'FINCH TO SHEPPARD WEST': (43.7812974, -79.4158993),
 'YORK MILLS CENTER TRAC': (43.7440391, -79.406657),
 'YONGE-UNIVERISTY AND B': (43.6541457, -79.38079276646786),
 'SHEPPARD STATION (APPR': (43.7493883, -79.4602908),
 'ST GEORGE TO LAWRENCE': (43.6684465, -79.398844),
 'MCCOWAN STATION (APPRO': (43.7749091, -79.2515777),
 'ST GEORGE AND BROADVIE': (43.6684465, -79.398844),
 'LINE 3 SCARBOROUGH RT': (43.6093021, -79.5493908),
 'SHEPPARD-YONGE TAIL TR': (43.733027, -79.5316661),
 'DUPONT STATION': (43.6747046, -79.406983),
 'KENNEDY BD STATION PLA': (43.62150235, -79.51232120817984),
 'WILSON YARD PLANT BUIL': (43.740426799999995, -79.4569393887856),
 'ALLEN RD (N OF EGLINTO': (43.7592761, -79.4650199),
 'BLOOR DANFORTH LINE ': (43.6443557, -79.527766),
 'VARIOUS': (43.6534817, -79.3839347),
 'KIPLING STATION (LEAVI': (43.63753245, -79.53560410315569),
 'YONGE STATION': (43.649114299999994, -79.37837180000002),
 'LAWRENCE TO ST CLAIR': (43.7263483, -79.4024743),
 'TETRA RADIO ZONE YU/SH': (None, None),
 'SPADINA TO KING STATIO': (43.6685404, -79.4047039),
 'HILLCREST - PAINT AND': (43.6816953, -79.4257118),
 'SOUTH OF MIDLAND SRT': (-33.013441, 151.5942043),
 'LINE 3': (43.6093021, -79.5493908),
 'KENNEDY TO VICTORIA PA': (43.732508, -79.2637953),
 'ELLESMERE STATION': (43.7669641, -79.2774794),
 'KEELE TO HIGH PARK': (43.655569, -79.4595795),
 'DANFORTH DIVISION': (43.6534817, -79.3839347),
 '120 CARTWRIGHT AVE': (43.72196327272727, -79.46690265454546),
 'KENNEDY BD STATION (EX': (43.732508, -79.2637953),
 'LAWRENCE EAST STATION': (43.7504987, -79.2704438),
 'MCCOWAN YARD': (43.7759581, -79.24528165482738),
 'SHEPPARD STATION': (43.7493883, -79.4602908),
 'ST GEORGE SIGNALS OFFI': (43.6684465, -79.398844),
 'OSSINGTON TO KEELE STA': (43.6534817, -79.3839347),
 'FINCH TO NORTH YORK CT': (43.7812974, -79.4158993),
 'ISLINGTON STATION (EXI': (43.6534817, -79.3839347),
 'FINCH TO LAWRENCE STAT': (43.7812974, -79.4158993),
 'YONGE-UNIVERSITY SUBW': (43.6541457, -79.38079276646786),
 'GUNN BUILDING': (43.6534817, -79.3839347),
 'BLOOR TO DAVISVILLE': (43.6577734, -79.4507397),
 'DONLANDS TO KIPLING ST': (43.6809856, -79.33781360458943),
 'VMC TO ST GEORGE STATI': (43.6534817, -79.3839347),
 'EASTBOUND - BETWEEN SH': (43.7627232, -79.2626354),
 'WILSON YARD - NORTH HO': (43.740426799999995, -79.4569393887856),
 'N/O DAVISVILLE TO N/O': (43.6534817, -79.3839347),
 'DUNDAS WEST STATION': (43.6569564, -79.4538893),
 'LAWRENCE EAST TO ELLES': (43.7504987, -79.2701863),
 'UNION TO ST CLAIR WEST': (43.6456424, -79.3802085),
 'BLOOR TO LAWRENCE STAT': (43.6577734, -79.4507397),
 'YONGE/UNIVERSITY': (43.6707855, -79.3856867),
 'SHERBOURNE STATION (EN': (43.6721875, -79.3764921),
 'GREENWOOD GATEHOUSE': (43.6534817, -79.3839347),
 'VICTORIA PARK': (43.66692875, -79.39195217788848),
 'KENNEDY BD STATION ( A': (43.732508, -79.2637953),
 'BLOOR TO MUSUEM STATIO': (43.6577734, -79.4507397),
 'BLOOR TO KING STATIONS': (43.6577734, -79.4507397),
 'YONGE/UINVERSITY & BLO': (43.7940038, -79.5273579),
 'DUNDAS STATION (W/S)': (43.6570301, -79.4529728),
 'BAY LOWER': (43.6705068, -79.3884643),
 'ST CLAIR WEST TO SHEPP': (43.6849986, -79.4157296),
 'YORK MILLS CENTRE': (43.74705725, -79.38504647563798),
 'ST CLAIR WEST TO KING': (43.6849986, -79.4157296),
 'SHEPPARD WEST MIGRATIO': (43.733027, -79.5316661),
 'LAWRENCE TO ST CLAIR S': (43.7263483, -79.4024743),
 'OLD MILL STATION (EXIT': (43.6501618, -79.495091),
 'SHEPPARD - YONGE STATI': (43.733027, -79.5316661),
 'KENNEDY - MCCOWAN': (43.732508, -79.2637953),
 'KIPLING STATION (ENTER': (43.63753245, -79.53560410315569),
 'DUPONT TO OSGOODE': (43.6747046, -79.406983),
 'YUS/BD/SEPPARD SUBWAY': (43.6534817, -79.3839347),
 'YONGE/UNIVERSITY LINE': (43.7432155, -79.4064144),
 'SHEPPARD WEST STATION': (43.7497578, -79.4624165),
 'ROYAL YORK STATION': (43.6481827, -79.5112961),
 'SPADINA TO OSGOODE': (43.6685404, -79.4047039),
 'MUSEUM STATION': (43.667097, -79.3936065),
 'WARDEN STATION (APPR': (43.7109797, -79.2797044),
 'ATC AREA': (-42.8656815, 147.4935427),
 'SCARBOROUGH RAPID TRAN': (43.7729744, -79.2576479),
 'KING STATION TO SPADIN': (43.649114299999994, -79.37837180000002),
 'EGLINTON TO KING STATI': (43.6534817, -79.3839347),
 'LEAVING DON MILLS': (43.775679, -79.34564065),
 'KENNEDY SRT STATION': (43.732508, -79.2637953),
 'BROADVIEW TO ST GEORGE': (43.6769999, -79.35823476332092),
 'WILSON YARD (SOUTH TAI': (43.740426799999995, -79.4569393887856),
 'NORTH YORK CENTRE STAT': (43.7543263, -79.44911696639593),
 'YONGE/UNIVERSITY-SPADI': (43.6707855, -79.3856867),
 'DAVISVILLE STATION': (43.6975459, -79.3968462),
 'WILSON TO SHEPPARD WES': (43.7344762, -79.4500619),
 'KIPLING TO KENNEDY STN': (43.6376161, -79.5355081),
 'LYTTON EMERGENCY EXIT': (43.6534817, -79.3839347),
 'YONGE UNIVERSITY BLOOR': (43.6707855, -79.3856867),
 'EARLY CLOSURE EGLINTON': (43.744189, -79.3139088),
 'YORK MILLS STATION (AP': (43.745179, -79.4061287),
 'WILSON COMPEX TRAINING': (43.7344762, -79.4500619),
 'CHRISTIE STATION': (43.6640809, -79.4185289),
 'DUNDAS STATION': (43.6570301, -79.4529728),
 'YONGE SHEP STATION': (43.6541457, -79.38079276646786),
 'WOODBINE STATION (EXIT': (43.6863861, -79.31271156664201),
 'PIONNER VILLAGE STATIO': (43.77708406639715, -79.51040430343632),
 'EGLINTON WEST TO VMC': (43.6534817, -79.3839347),
 'DOWNSVIEW PARK STATION': (43.7536301, -79.4781785),
 'WILSON TO ST CLAIR WES': (43.7344762, -79.4500619),
 'MC BRIEN BUILDING': (43.69760659809708, -79.39708678994454),
 'YONGE SHEPPARD SATATIO': (43.6541457, -79.38079276646786),
 'SHEPPARD/YONGE STATION': (43.7614518, -79.4109148),
 'YONGE/UNIVERSITY/ BLOO': (43.6534817, -79.3839347),
 'OSSINGTON STATION (EXI': (43.6534817, -79.3839347),
 'VMC TO PIONEER VILLAGE': (43.6534817, -79.3839347),
 'BAYVIEW TO SHEPAPRD/YO': (43.7667987, -79.3873787),
 'SCARBOROUGH RAPID TRA': (43.7729744, -79.2576479),
 'HIGH PARK STATION (APP': (43.6540242, -79.4663199),
 'MCCOWAN CARHOUSE': (43.6534817, -79.3839347),
 'FICNH STATION': (43.6534817, -79.3839347),
 'WILSON CAPITAL SHOP': (43.7344762, -79.4500619),
 'KENNEDY SRT TO MCCOWA': (43.6534817, -79.3839347),
 'WELLESLEY STATION': (43.665403, -79.3836001),
 'YONGE UNVERSITY SUBWAY': (43.6541457, -79.38079276646786),
 'YONGE SHP STATION (LEA': (43.6541457, -79.38079276646786),
 'SHEPPARD LINE': (43.7668565, -79.3873063),
 'FINCH TO DOWNSVIEW': (43.7812974, -79.4158993),
 'ST CLAIR STATION (APPR': (43.6879575, -79.3918194),
 'ST ANDREW STATION': (43.6475462, -79.3854279),
 'ST GEORGE BEDFORD': (43.6686081, -79.398161),
 'WILSON HOSTLER': (43.6534817, -79.3839347),
 'UNION, EXHIBITION, MIM': (43.6456424, -79.3802085),
 'VMC STATION TO PIONEER': (43.6534817, -79.3839347),
 'QUEEN STATION - EAST S': (43.6529083, -79.3794575),
 'DAVISVILLE YARD - T&S': (43.6959713, -79.39732267921579),
 'DUNDAS STATION E/S': (43.6565367, -79.3810223),
 'DONLANDS STATION (APPR': (43.6809856, -79.33781360458943),
 'ASQUITH SUBSTATION': (43.6534817, -79.3839347),
 'YORK MILLS CENTRE TRAC': (43.7440391, -79.406657),
 'BATHUSRT STATION': (43.6534817, -79.3839347),
 'ST CLAIR TO LAWRENCE S': (43.6879512, -79.3936947),
 'ST ANDREW TO SPADINA': (43.6477917, -79.3848711),
 'WOODBINE TO ST.GEORGE': (43.6863861, -79.31271156664201),
 'KENNEDY STATION (PALTF': (43.7323896, -79.2637081),
 'KENNEDY BD TO KIPLING': (43.62150235, -79.51232120817984),
 'NORTH HOSTLER': (43.6534817, -79.3839347),
 'BROADVIEW AND DANFORTH': (43.6769999, -79.35823476332092),
 'YONGE-UNIVERSITY AND B': (43.6541457, -79.38079276646786),
 'ST GEORGE STATION TO G': (43.6686414, -79.39801880470816),
 'YOONGE-UNIVERSITY SUBW': (43.6534817, -79.3839347),
 'INGLIS BUILDING - 3RD': (43.7878432, -79.4536805),
 'SHEPPARD WEST TO ST. C': (43.7492988, -79.462248),
 'CHANGE OVERS / GENERAL': (None, None),
 'LAWRENCE STATION (LEAV': (43.7250043, -79.4022502),
 'S/O EGLINTON TO ST CLA': (43.6534817, -79.3839347),
 'ST. CLAIR STATION': (43.6879575, -79.3918194),
 'BATHURST STATION': (43.6665624, -79.4112998),
 'YONGE AND BLOOR': (43.6541457, -79.38079276646786),
 'GLENAYR EMERGENCY EXIT': (43.6962511, -79.4197343),
 'UNION STATION': (43.6447302, -79.38005624881814),
 'MUSEUM STATION (LEAVIN': (43.667097, -79.3936065),
 'GREENWOOD STATION': (43.68253285, -79.33024260293422),
 'WILSON STATION (EXITIN': (43.7347436, -79.4508641),
 'YORK MILLS STATION (A': (43.745179, -79.4061287),
 'ISLINGTON STATION': (43.6534817, -79.3839347),
 'KING STATION TO EGLINT': (43.649114299999994, -79.37837180000002),
 'OSSINGTON CENTRE TRACK': (43.6534817, -79.3839347),
 'LESLIE STATION': (43.7705457, -79.3678254),
 'YU/DB/SHEPPARD SUBWAY': (43.6534817, -79.3839347),
 'OSGOODE POCKET': (43.6534817, -79.3839347),
 'YORK UNIVERSITY STATIO': (43.6896191, -79.479188),
 'WILSON - ST CLAIR WEST': (43.7344762, -79.4500619),
 'WARDEN TO KENNEDY STAT': (43.7111204, -79.2790432),
 'YONGE UNIVERSTY SUBWAY': (43.6541457, -79.38079276646786),
 'KEELE YARD': (43.6534817, -79.3839347),
 'EGLINTON STATION': (43.6534817, -79.3839347),
 'YARD SWITCH - BD CHECK': (43.6534817, -79.3839347),
 'EGLINTON WEST TO BLOOR': (43.6534817, -79.3839347),
 'EGLINTON STATION (EXIT': (43.6534817, -79.3839347),
 'YONGE TO COXWELL STATI': (43.6541457, -79.38079276646786),
 'CLANTON PARK PORTAL': (43.6534817, -79.3839347),
 'YONGE UNIVERSITY AND B': (43.6707855, -79.3856867),
 'S/O SHEPPARD TO DAVISV': (43.6534817, -79.3839347),
 'YONGE-UNIVERSITY AND': (43.6541457, -79.38079276646786),
 'KING TO SPADINA STATIO': (43.6489494, -79.3777538),
 'OLD MILL STATION (APPR': (43.6501618, -79.495091),
 'KEELE TO HIGH PARK STA': (43.655569, -79.4595795),
 'SHEPPARD WEST PORTAL': (43.733027, -79.5316661),
 'YONGE - SHEPPARD (LEAV': (43.6541457, -79.38079276646786),
 'DAVISVILLE BUILD UP': (43.697936, -79.3972908),
 'KENNEDY SRT TO MCCOWAN': (43.6534817, -79.3839347),
 'ST CLAIR WEST TO DUPON': (43.6849986, -79.4157296),
 'NORTHBOUND TO LAWRENCE': (43.665113, -79.385511),
 'YONGE UNIVERSITY/BLOOR': (43.6707855, -79.3856867),
 'YONGE - UNIVERSITY SUB': (43.6541457, -79.38079276646786),
 'VICTORIA PARK STATION': (43.66692875, -79.39195217788848),
 'COLLEGE STATION': (43.6606617, -79.3827952),
 'HIGH PARK STATION (ENT': (43.6540242, -79.4663199),
 'DANFORTH AND COXWELL': (43.6864333, -79.3003555),
 'MC COWAN STATION': (43.774853610113574, -79.25200049179068),
 'TRANSIT CONTROL': (43.6534817, -79.3839347),
 'MAIN STREET AND UNION': (43.689040750000004, -79.30162111836233),
 'YONGE UNIVERSITY SPADI': (43.6541457, -79.38079276646786),
 'ST GEORGE TO BAY STATI': (43.6684465, -79.398844),
 'YONGE-UNIVERSITY SPADI': (43.6541457, -79.38079276646786),
 'SHEPPARD YONGE': (43.7614518, -79.4109148),
 'ST.CLAIR WEST TO ST.A': (43.6849986, -79.4157296),
 'SCARBORUOGH RAPID TRAN': (43.7167253730465, -79.25482778994406),
 'DON MLLS STATION': (43.6680266, -79.3692816),
 'YONGE/UNIVERSITY & BLO': (43.6707855, -79.3856867),
 'DAVISVILLE STATION - L': (43.6975459, -79.3968462),
 'VAUGHAN MC STATION': (43.68285675, -79.41953968088616),
 'BLOOR TO EGLINTON': (43.6577734, -79.4507397),
 'KING AND BATHURST': (43.64124975, -79.40938495),
 'LAWRENCE STATION TO YO': (43.7250043, -79.4022502),
 'SHEPPARD WEST': (43.7492988, -79.462248),
 'OSSINGTON STATION (APP': (43.6534817, -79.3839347),
 'ST. CLAIR TO FINCH STA': (43.6879512, -79.3936947),
 'ST GEORGE TO BLOOR STA': (43.6684465, -79.398844),
 'MARKDALE EE': (44.3217734, -80.65331),
 'KENNEDY SRT STATION (E': (43.732508, -79.2637953),
 'KIPLING TO COXWELL': (43.6376161, -79.5355081),
 'EGINTON STATION': (43.6534817, -79.3839347),
 'SUBWAY CLOSURE - WILSO': (43.6534817, -79.3839347),
 'FINCH TO SHEPPARD STAT': (43.7812974, -79.4158993),
 'TYSSE LINE': (43.6534817, -79.3839347),
 'WILSON DIVISION': (43.6534817, -79.3839347),
 'GLENAYR SUBSTATION': (43.6534817, -79.3839347),
 'EGLINTON TO DAVISVILLE': (43.6534817, -79.3839347),
 'PAPE TO ST GEORGE': (43.67996425, -79.34474097964232),
 'DUPONT STATION (APPROA': (43.6747046, -79.406983),
 'DUNDAS WEST STATION (A': (43.6569564, -79.4538893),
 'DAVISVILLE TO EGLINTON': (43.697936, -79.3972908),
 'QUEENS PARK STATION': (46.4524682, -63.3799629),
 'ROSEDALE STATION (APPR': (43.676735050000005, -79.3886162498014),
 'LEAVING KENNEDY STATIO': (43.73238561230919, -79.26383038624765),
 'YONGE-UNIVERSITY/BLOOR': (43.6541457, -79.38079276646786),
 'JANE STATION (EXITING)': (43.6499177, -79.4838463),
 'ATC FAILURE': (43.6534817, -79.3839347),
 'GREENWOOD STATION (APP': (43.68253285, -79.33024260293422),
 'HIGH PARK TO KEELE': (43.6538668, -79.4668644),
 'BLOOR SATION': (43.6534817, -79.3839347),
 'YONGE- UNIVERSITY AND': (43.6541457, -79.38079276646786),
 'KENNEDY STATION TO WOO': (43.7323896, -79.2637081),
 'QUEENS QUAY STATION': (46.4524682, -63.3799629),
 'DUNDAS STATION E/S 1': (43.6570301, -79.4529728),
 'KIPLING TO KENNEDY STA': (43.6376161, -79.5355081),
 'KIPLING STATION (PLATF': (43.63753245, -79.53560410315569),
 'YONGE/UNIVESITY AND BL': (43.6541457, -79.38079276646786),
 'LINE 4': (43.6311589, -79.3571504),
 'SHEPPARD YONGE STATION': (43.7614518, -79.4109148),
 'KING STATION TO ST GEO': (43.649114299999994, -79.37837180000002),
 'KIPLING TO HIGH PARK': (43.6376161, -79.5355081),
 'ST. PATRICK STATION': (43.6548199, -79.3882736),
 'MOORE GATE': (43.6534817, -79.3839347),
 'ST GEORGE STATION TO B': (43.6686414, -79.39801880470816),
 'ST. CLAIR WEST STATION': (43.6841761, -79.4155252),
 'YONGE UNIVERSITY SUBWA': (43.6541457, -79.38079276646786),
 'BLOOR TO EGLINTON STAT': (43.6577734, -79.4507397),
 'YONGE AND SHEPPARD': (43.6541457, -79.38079276646786),
 'DUPONT STATION (LEAVIN': (43.6747046, -79.406983),
 'FINCH STATION (APPROAC': (43.7827501, -79.41488829825582),
 'BROADVIEW TO YONGE STA': (43.6769999, -79.35823476332092),
 'GREENWOOD PORTAL': (43.6534817, -79.3839347),
 'TRANSIT CONTROL CENTRE': (43.6977732, -79.39690388301321),
 'EGLINTON STATION TO FI': (43.6534817, -79.3839347),
 'DON MILLS STATION': (43.775725, -79.3452775),
 'YONGE UNIVERSITY LINE': (43.7432155, -79.4064144),
 'SHEPPARD - EGLINTON ST': (43.733027, -79.5316661),
 '40 CARL HALL ROAD UNIT': (43.750224, -79.47935176133203),
 'KEELE STATION': (43.6554366, -79.4596039),
 'ST CLAIR TO BLOOR STAT': (43.6879512, -79.3936947),
 'ST GEORGE BD STATION -': (43.6684465, -79.398844),
 'WARDEN TO VICTORIA PAR': (43.7111204, -79.2790432),
 'WILSON STATION (APPROA': (43.7347436, -79.4508641),
 'SHEPPARD WEST TO ST CL': (43.7492988, -79.462248),
 'YONGE - UNIVERSITY BLO': (43.6541457, -79.38079276646786),
 'WARDEN TO KENNEDY': (43.7111204, -79.2790432),
 'SHEPPARD SUBWAY LINE': (43.733027, -79.5316661),
 'KIPLING TO KENNEDY': (43.6376161, -79.5355081),
 'DON MILLS STAITON': (43.6680266, -79.3692816),
 'WILSON YARD': (43.740426799999995, -79.4569393887856),
 'CHESTER TO BROADVIEW': (43.6782277, -79.3525143),
 'SHEPPARD STATION (ENTE': (43.7493883, -79.4602908),
 'BLOOR - DANFORTH LINE': (43.6577734, -79.4507397),
 'LESLIE STATION APROACH': (43.7710725, -79.365634),
 'EGLINTON BUS TERMINAL': (43.6534817, -79.3839347),
 'CHESTER TO YONGE STATI': (43.6782277, -79.3525143),
 'BROADVIEW STATION': (43.6770471, -79.35839530606879),
 'BLOOR TO OSGOODE STATI': (43.6577734, -79.4507397),
 'SHEPPARD STATION TO YO': (43.7493883, -79.4602908),
 'UNION STATION (DOWNSVI': (43.6447302, -79.38005624881814),
 'ST GEORGE STN': (43.6686414, -79.39801880470816),
 'ATC TERRITORY': (43.6534817, -79.3839347),
 'BICHMOUNT DIVISION': (43.6534817, -79.3839347),
 'BLOOR TO ST.CLAIR': (43.6577734, -79.4507397),
 'GREENWOOD - TRACK TRAI': (43.68253285, -79.33024260293422),
 'ROYAL YORK STATION (AP': (43.6481827, -79.5112961),
 'GREENWOOD WYE': (43.6534817, -79.3839347),
 'OSSINGTION STAITON': (43.6534817, -79.3839347),
 'SHEPPARD WYE': (43.6534817, -79.3839347),
 'SUBWAY OPERATIONS BUIL': (43.764473, -79.4120812),
 'SPADINA STATION YUS': (43.6685404, -79.4047039),
 'KENNEDY AND SCARBOROUG': (43.732508, -79.2637953),
 'CASTLE FRANK STATION': (43.673509, -79.36903055286996),
 'KIPLING TO ISLINGTON': (43.6376161, -79.5355081),
 'ST CLAIR WEST TO UNION': (43.6849986, -79.4157296),
 'KENNEDY STATION TO KIP': (43.7323896, -79.2637081),
 'HIGH PARK STATION': (43.6540242, -79.4663199),
 'YONGE-UNIVERSITY SUBWA': (43.6541457, -79.38079276646786),
 'UNION TO KING': (43.6456424, -79.3802085),
 'YONGE- UNIVERSITY SUBW': (43.6541457, -79.38079276646786),
 'SHEPPARD TO BAYVIEW': (43.733027, -79.5316661),
 'WILSON TO LAWRENCE WES': (43.7344762, -79.4500619),
 'GREENWOOD WYE (ENTERI': (43.6534817, -79.3839347),
 'UNION STATION TO KING': (43.6447302, -79.38005624881814),
 'ST CLAIR STATION (LEAV': (43.6879575, -79.3918194),
 'LINE 2 (BD)': (43.6127719, -79.3879488),
 'KIPLING - KENNEDY': (43.6376161, -79.5355081),
 "QUEEN'S PARK STATION": (43.659659, -79.3903399),
 'GUNN THEATRE': (43.6534817, -79.3839347),
 'PLANT OPS BLDG': (43.6384267, -79.38201308844691),
 'FINCH TO UNION STATION': (43.7812974, -79.4158993),
 'ST. ANDREW STATION': (43.6475462, -79.3854279),
 'MCCOWAN STATION NB': (43.7749091, -79.2515777),
 'UNION TO MAIN STREET': (43.6456424, -79.3802085),
 'SPADINA BD STATION': (43.6685404, -79.4047039),
 'ISLINGTON STATION (APP': (43.6534817, -79.3839347),
 'UNION STATION - BOOTH': (43.6447302, -79.38005624881814),
 'ST. GEORGE TO BROADVIE': (43.6684465, -79.398844),
 '(APPROACHING)': (43.6534817, -79.3839347),
 'SHERBOURNE STATION (AP': (43.6721875, -79.3764921),
 'ST GEORGE STATION': (43.6686414, -79.39801880470816),
 'DAVISVILLE HOSTLER': (43.6534817, -79.3839347),
 'VICTORIA PARK TO WARDE': (43.66692875, -79.39195217788848),
 'DUNDAS STATION - EAST': (43.6570301, -79.4529728),
 'SHEPPARD WEST TO LAWRE': (43.7492988, -79.462248),
 'SRT LINE': (43.6534817, -79.3839347),
 'LAWRENCE WEST TO EGLIN': (43.7152827, -79.4439143),
 'CHESTER STATION': (43.6782277, -79.3525143),
 'SUBWAY CLOSURE - JANE': (43.6534817, -79.3839347),
 'BLOOR TO YORK MILLS': (43.6577734, -79.4507397),
 'DAVENPORT BUILDING': (43.6612461, -79.39776281717747),
 'LOWER BAY': (43.6706718, -79.3880299),
 'YORKDALE STATION (EXIT': (43.7246418, -79.4475031),
 'CLANTON PARK EMERGENCY': (43.6534817, -79.3839347),
 '169 DANFORTH AVENUE': (43.6851302, -79.3153548),
 'WELBECK EMERGENCY EXIT': (43.7595037, -79.4208297),
 'UNION STATION TO ST CL': (43.6447302, -79.38005624881814),
 'LAWRENCE STATION': (43.7250043, -79.4022502),
 'FINCH WEST STATION': (43.7639431, -79.4904358),
 'GO PROTOCOL - BARRIE L': (43.6534817, -79.3839347),
 'WILSON YARD NORTH HOST': (43.740426799999995, -79.4569393887856),
 'BROADVIEW TO SHERBOURN': (43.6769999, -79.35823476332092),
 'WARDEN AND KENNEDY': (43.7111204, -79.2790432),
 'GREENWOOD PORTAL (EXIT': (43.6534817, -79.3839347),
 'FINCH STATION (EXITING': (43.7827501, -79.41488829825582),
 'CLOSURE- KEELE TO OSSI': (43.6534817, -79.3839347),
 'BLOOR SOUTH': (43.6669994, -79.3997021),
 'TORONTO TRANSIT CONTRO': (43.6534817, -79.3839347),
 'EGLINTON STATION (LEAV': (43.6534817, -79.3839347),
 'QUEEN TO FINCH STATION': (43.6529083, -79.3794575),
 'WILSON CAR HOUSE': (43.7344762, -79.4500619),
 'ST CLAIR WEST TO EGLIN': (43.6849986, -79.4157296),
 'SPADINA TO UNION STATI': (43.6685404, -79.4047039),
 'BLOOR DANFORTH/YONGE U': (43.6706654, -79.3871168),
 'BROADVIEW TO KIPLING S': (43.6769999, -79.35823476332092),
 'BLOOR TO SHEPPARD STAT': (43.6577734, -79.4507397),
 'FINCH - NORTH YORK CEN': (43.7812974, -79.4158993),
 'COXWELL TO KENNEDY STA': (43.68414715, -79.32303491486606),
 'FINCH TO EGLINTON': (43.7812974, -79.4158993),
 'SHEPPARD WEST TO DOWNS': (43.7492988, -79.462248),
 'JANE STATION (APPROACH': (43.6499177, -79.4838463),
 'N/O YORK MILLS TO DAVI': (43.7331878, -79.3396107),
 'ISLINGTON STATION (ENT': (43.6534817, -79.3839347),
 'DAVISVILLE BUILDUP': (43.6534817, -79.3839347),
 'UNION AND KENNEDY STAT': (43.6614803, -79.3829401),
 'KIPLING TO JANE': (43.6376161, -79.5355081),
 'RAMPS BAY LOWER': (43.67031405825204, -79.38997184392011),
 'DUPONT TO ST CLAIR WES': (43.6747046, -79.406983),
 'DUFFERIN TO LANSDOWNE': (43.6602019, -79.4357191),
 'KENNEDY - WARDEN - MAI': (43.732508, -79.2637953),
 'QUEEN STATION': (43.6529083, -79.3794575),
 'DUPONT STATION ( APPRO': (43.6747046, -79.406983),
 'MCCOWAN STATION': (43.7749091, -79.2515777),
 'SUMMERHILL STATION': (43.6816776, -79.3905037),
 'SPADINA STATION TO KIN': (43.6672349, -79.4036863),
 'SPADINA TO ST ANDREW': (43.6685404, -79.4047039),
 'BAY LOWER STATION': (43.6701851, -79.3902694),
 'SHEPPARD WEST - WILSON': (43.7492988, -79.462248),
 'UNION TO FINCH STATION': (43.6456424, -79.3802085),
 'WILSON HOSTLER ': (43.6534817, -79.3839347),
 'SYSTEMWIDE': (43.6534817, -79.3839347),
 'BLOOR-DANFORTH SUBWAY': (43.6577734, -79.4507397),
 'KENNEDY SRT STATION (A': (43.732508, -79.2637953),
 'KIPLING TO JANE STATIO': (43.6376161, -79.5355081),
 'YONGE/UNIVERSITY AND B': (43.6707855, -79.3856867),
 'KEELE STATION (ENTERIN': (43.6554366, -79.4596039),
 'VAUGHAN MC STATIO': (43.68285675, -79.41953968088616),
 'LAWERENCE STATION': (43.6534817, -79.3839347),
 'SUBWAY OPS BUILDING': (43.764473, -79.4120812),
 'WOODBINE STATION': (43.6863861, -79.31271156664201)}